<a href="https://colab.research.google.com/github/GIBD/PS-News/blob/master/1_UTN_Breaking_News_RAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 - UTN -  Breaking News

In [0]:
import pandas as pd
import os
import chardet
import numpy as np
import datetime
from os import path

In [2]:
! git clone https://github.com/GIBD/PS-News.git

Cloning into 'PS-News'...
remote: Enumerating objects: 582, done.
remote: Total 582 (delta 0), reused 0 (delta 0), pack-reused 582
Receiving objects: 100% (582/582), 45.25 MiB | 20.55 MiB/s, done.
Resolving deltas: 100% (22/22), done.
Checking out files: 100% (568/568), done.


In [30]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
os.chdir("/content/PS-News")

In [17]:
DIRECTORIO_BASE = "/content/PS-News"
DIRECTORIO_DATA = "/content/PS-News/data/raw/2020_03"
DIRECTORIO_EXPORT = "/content/PS-News/export"

'/content/PS-News/export'

In [18]:
DIRECTORIO_BASE

'/content/PS-News'

In [19]:
DIRECTORIO_DATA

'/content/PS-News/data/raw/2020_03'

In [20]:
DIRECTORIO_EXPORT

'/content/PS-News/export'

In [21]:
ls

data/  export/  README.md


In [0]:
# Constantes

# Solamente es necesario cambiar el directorio base

#DIRECTORIO_BASE = "/content/PS-News"
#DIRECTORIO_DATA = os.chdir("/content/PS-News/data/raw/2020_03")
#DIRECTORIO_EXPORT = path.join(DIRECTORIO_BASE, 'export')

TEMPLATE_EXPORT = 'news_marzo'

MES = 3
DIA_INICIO = 1
DIA_FIN = 31

DIARIOS = ['clarin','infobae','lanacion'] # Metadatos para acceder a los archivos JSON
SECCIONES = ['deportes','economia','sociedad','tecnologia', 'politica'] # elimine la seccion 'politica' porque faltan de INFOBAE
DIAS = list(range(DIA_INICIO, DIA_FIN + 1))


In [0]:
# Utilidades

def generar_nombre_export(nombre_export):
    formato = "%Y%m%d-%H%M%S"
    fecha_actual = datetime.datetime.now().strftime(formato)
    nombre_archivo = f'{TEMPLATE_EXPORT}_{nombre_export}_{fecha_actual}.csv'
    
    return path.join(DIRECTORIO_EXPORT, nombre_archivo)

def log(mensaje, nivel = 0):
    tabs = ('\t' * nivel) if nivel is not 0 else ''
    print(tabs + mensaje)
    
def crear_directorio_si_no_existe(directorio):
    if not path.exists(directorio):
        os.makedirs(directorio)


In [24]:


crear_directorio_si_no_existe(DIRECTORIO_BASE)
crear_directorio_si_no_existe(DIRECTORIO_EXPORT)

os.chdir(DIRECTORIO_DATA)     

# Inicializa dataframe noticias
noticias = pd.DataFrame()

# Recorre los archivos JSON cargando las noticias en el dataframe noticias
for diario in DIARIOS:
    for seccion in SECCIONES:
        for dia in DIAS:
            
            # Genera el nombre del JSON a procesar
            nombreJSON = f'{diario}_{seccion}_2020{MES:02d}{dia:02d}.json'
            
            log(f'Procesando archivo {nombreJSON}:')

            # Genera la fecha del JSON actual
            fechaJSON = datetime.datetime(2020, MES, dia)
            
            if not path.exists(nombreJSON):
                log('Archivo no encontrado.', 1)
                continue
            
            # Obtiene metadatos del archivo para acceder al encoding
            with open(nombreJSON, 'rb') as f: 
                result = chardet.detect(f.read())
            
            # Obtiene el dataframe a partir del JSON teniendo en cuenta el encoding
            encoding = 'utf-8' if result['encoding'] != 'Windows-1252' else result['encoding']
            
            # convert_dates en False es porque:
            #    1) los archivos de infobae (ej infobae_economia_20200317) tienen un array en la llave date
            #    2) igualmente la fecha se pisa en el siguiente paso, asi que tampoco importa
            df = pd.read_json(nombreJSON, lines=True, encoding=encoding, convert_dates=False)
                
            # Actualiza la fecha en el dataframe
            df['date'] = fechaJSON
            
            #concatena con los anteriores
            noticias = pd.concat([noticias, df], ignore_index=True, sort=False)
            
            log('Archivo procesado correctamente.', 1)          


Procesando archivo clarin_deportes_20200301.json:
	Archivo no encontrado.
Procesando archivo clarin_deportes_20200302.json:
	Archivo no encontrado.
Procesando archivo clarin_deportes_20200303.json:
	Archivo no encontrado.
Procesando archivo clarin_deportes_20200304.json:
	Archivo procesado correctamente.
Procesando archivo clarin_deportes_20200305.json:
	Archivo procesado correctamente.
Procesando archivo clarin_deportes_20200306.json:
	Archivo procesado correctamente.
Procesando archivo clarin_deportes_20200307.json:
	Archivo procesado correctamente.
Procesando archivo clarin_deportes_20200308.json:
	Archivo procesado correctamente.
Procesando archivo clarin_deportes_20200309.json:
	Archivo procesado correctamente.
Procesando archivo clarin_deportes_20200310.json:
	Archivo procesado correctamente.
Procesando archivo clarin_deportes_20200311.json:
	Archivo procesado correctamente.
Procesando archivo clarin_deportes_20200312.json:
	Archivo procesado correctamente.
Procesando archivo cla

In [25]:
# Exporta el dataframe de noticias en formato CSV
export_csv = noticias.to_csv(generar_nombre_export('todas'),index = None, header=True)

#noticias.shape
noticias

,news_paper,category,link,date,title,abstract,author,key_words,body
0,clarin,deportes,https://www.clarin.com//deportes/velez-sarsfie...,2020-03-04,"Vélez Sarsfield vs Godoy Cruz, en vivo por la ...",El Fortín busca afianzarse en el tercer lugar ...,[],Superliga ArgentinaClub Atlético Vélez Sarsfie...,Vélez Sarsfield recibe a Godoy Cruz de Mendoza...
1,clarin,deportes,https://www.clarin.com//deportes/zielinski-cam...,2020-03-04,Cómo se prepara Atlético Tucumán para frenar a...,El “Ruso” Zielinski analiza variantes en su on...,[],Atlético TucumánRiver PlateSuperliga Argentina,Atlético Tucumán no quiere ser espectador de l...
2,clarin,deportes,https://www.clarin.com//deportes/liga-de-quito...,2020-03-04,"Liga de Quito vs River, en vivo por Copa Liber...",El equipo argentino afronta el debut en la Cop...,[],Copa LibertadoresRiver PlateMarcelo GallardoLi...,"River Plate visita a Liga de Quito, en uno de ..."
3,clarin,deportes,https://www.clarin.com//deportes/ponzio-quito-...,2020-03-04,"Ponzio, en Quito: “Hay que aprovechar nuestras...","El capitán, que será titular ante Liga, remarc...",Maximiliano Uria,River PlateLeonardo PonzioCopa Libertadores,Celebrará de una manera especial Leonardo Ponz...
4,clarin,deportes,https://www.clarin.com//deportes/neymar-barcel...,2020-03-04,La carta que jugará Neymar para irse del PSG y...,Como el brasileño no tiene fijada una cifra de...,[],NeymarFC BarcelonaParis Saint Germain,La relación entre Neymar y el PSG<U+200B> camb...
...,...,...,...,...,...,...,...,...,...
22236,lanacion,politica,http://www.lanacion.com.ar//politica/las-princ...,2020-03-31,Coronavirus en la Argentina: las principales f...,None,,,El presidente \r\n\r\nAlberto Fernández\r\n\r\...
22237,lanacion,politica,http://www.lanacion.com.ar//politica/coronavir...,2020-03-31,Coronavirus: la Justicia rechazó la mayoría de...,None,Candela Ini,,La \r\n\r\njurisprudencia \r\n\r\nsobre el \r\...
22238,lanacion,politica,http://www.lanacion.com.ar//politica/conurbano...,2020-03-31,Coronavirus: Una travesía por los bloqueos dur...,None,Javier Fuego Simondet,,Con una enorme bolsa blanca y en medio de un n...
22239,lanacion,politica,http://www.lanacion.com.ar//politica/coronavir...,2020-03-31,Coronavirus: ¿Cómo y cuándo salir de la cuaren...,None,"[Eduardo Levy Yeyati, Andrés Malamud]",,"La pandemia es una crisis global, pero las res..."


---

**Resumen de cantidad de noticias según diario y sección**

---

In [0]:
# Agrupa las noticias por diario y categoría
a = noticias.groupby(['news_paper', 'category'])

In [0]:
# Genera un array numpay solo con la cantidad de noticias
b = np.array(a.aggregate(np.count_nonzero)['link'])

In [28]:
# Agrega al array la sumatoria por secciones

b = np.append(b,[b[0:5] + b[5:10] + b[10:15]])

# Genera un dataframe con la cantidad de noticias por seccion y diario y los totales

resumen = pd.DataFrame({"Clarin":np.append(b[0:5],[b[0:5].sum()]), 
                        "Infobae":np.append(b[5:10],[b[5:10].sum()]),  
                        "La Nacion":np.append(b[10:15],[b[10:15].sum()]),
                        "TOTAL":np.append(b[15:20],[b[15:20].sum()])},
             index=['Deportes', 'Economía','Politica','Sociedad','Tecnologia','TOTAL'])

# Muestra el dataframe
resumen.T

,Deportes,Economía,Politica,Sociedad,Tecnologia,TOTAL
Clarin,924,924,941,924,924,4637
Infobae,3121,975,860,1680,2100,8736
La Nacion,1279,1077,2372,2070,2070,8868
TOTAL,5324,2976,4173,4674,5094,22241


---

Se exporta la tabla resumen como archivo CSV

---

In [0]:
news_resumen = resumen.to_csv(generar_nombre_export('resumen'), index=True, header=True)